Run the following line if you don't have any of the libraries

In [ ]:
!pip install statsmodels pandas numpy --upgrade --user

In [24]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.tsa.ar_model import AutoReg, ar_select_order
from statsmodels.tsa.api import acf, pacf, graphics
import data_prep
import numpy as np

In [ ]:
data_prep.get_lagged("./data/XLE_energy_etf.csv", n=3)

In [ ]:
pd.read_csv("./data/SnP_500.csv", index_col="Date")

In [2]:
df = data_prep.get_perc_return("./data/SnP_500.csv", column_name="Adj Close")

In [12]:
df = df.dropna()
df.isna().sum()
df['Date'] = pd.to_datetime(df['Date'], infer_datetime_format=True)

In [23]:
indexeddf = df.set_index(['Date'])
indexeddf.head()

,%_returns
Date,
2010-01-05,0.003116
2010-01-06,0.000546
2010-01-07,0.004001
2010-01-08,0.002882
2010-01-11,0.001747


In [20]:
ar_1 = AutoReg(indexeddf, lags =1).fit()

C:\Users\Hamu\AppData\Roaming\Python\Python37\site-packages\statsmodels\tsa\base\tsa_model.py:218: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


In [25]:
ar_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            AutoReg Model Results                             
==============================================================================
Dep. Variable:              %_returns   No. Observations:                 2579
Model:                     AutoReg(1)   Log Likelihood                8053.771
Method:               Conditional MLE   S.D. of innovations              0.011
Date:                Fri, 03 Apr 2020   AIC                             -9.084
Time:                        16:57:18   BIC                             -9.077
Sample:                             1   HQIC                            -9.081
                                 2579                                         
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
intercept        0.0004      0.000      2.024      0.043    1.35e-05       0.001
%_returns.L1    -0.1557      0.019     -7.998      0.000      -0.194      -0.118
                                    Roots                                    
=============================================================================
                  Real          Imaginary           Modulus         Frequency
-----------------------------------------------------------------------------
AR.1           -6.4214           +0.0000j            6.4214            0.5000
-----------------------------------------------------------------------------
"""